# Data avalysis

### Avaliando e buscando Insights:



1. Quanto foi pago de despesas durante o mês?
2. Qual foi o orgão superior que mais teve despesas?
3. Quem mais se favoreceu com esses despesas do governo?
4. Quanto foi gasto com investimentos?

Esses foram apenas alguns dos muitos pontos que podem ser respondidos com essa análise de nossos dados.

In [33]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [2]:
# Abrindo dataframe
df = pd.read_csv('pagamentos_janeiro.csv')
df.head()

,Data_emissao,Tipo_documento,Tipo_ob,Extraorcamentario,Orgao_superior,Orgao,Unidade_gestora,Gestao,Favorecido,Observacao,Processo,Categoria_de_despesa,Grupo_de_despesa,Modalidade_de_aplicacao,Elemento_de_despesa,Valor_original_do_pagamento,Valor_do_pagamento_convertido_pra_r$,Valor_utilizado_na_conversao
0,01/01/2019,Nota de Lançamento de Sistema,Não se aplica,NÃO,MINISTERIO DA EDUCACAO,HOSPITAL DE CLINICAS DE PORTO ALEGRE,HOSPITAL DE CLINICAS DE PORTO ALEGRE,HOSPITAL DE CLINICAS DE PORTO ALEGRE,MARCOS VINICIUS ZINN HUBNER,APROP DA PROPOSTA DE CONCESSAO SF DE MARCOS VI...,Sem processo,DESPESAS CORRENTES,OUTRAS DESPESAS CORRENTES,APLICACOES DIRETAS,Múltiplo,2000.00,2000.00,1.0
1,02/01/2019,Ordem Bancária,OBC/OBB PARA TERCEIROS EM OUTROS BANCOS,NÃO,MINISTERIO DA CIDADANIA,MINISTERIO DA CIDADANIA - UNIDADES COM VÍNCUL,CEF/MINISTERIO DO ESPORTE,TESOURO NACIONAL,MUNICIPIO DE BREJINHO,OF ESPORTE 641 DE 28/12/2018 CONTRATO: CR.NR...,Sem processo,DESPESAS DE CAPITAL,INVESTIMENTOS,TRANSFERENCIAS A MUNICIPIOS,AUXILIOS - FUNDO A FUNDO,48750.00,48750.00,1.0
2,02/01/2019,Documento de Arrecadação Federal,Não se aplica,NÃO,"MINIST. DA AGRICUL.,PECUARIA E ABASTECIMENTO",SERVICO FLORESTAL BRASILEIRO,SERVICO FLORESTAL BRASILEIRO,TESOURO NACIONAL,SERPRO - SEDE - BRASILIA,"RETENCAO DE 9,45 POR CENTO REFERENTE A IN SRF ...",02209001806201841,DESPESAS CORRENTES,OUTRAS DESPESAS CORRENTES,APLICACOES DIRETAS,OUTROS SERVICOS DE TERCEIROS-PESSOA JURIDICA,50.03,50.03,1.0
3,02/01/2019,Ordem Bancária,OBC/OBB PARA TERCEIROS EM OUTROS BANCOS,NÃO,MINISTERIO DA CIDADANIA,MINISTERIO DA CIDADANIA - UNIDADES COM VÍNCUL,CEF/MINISTERIO DO ESPORTE,TESOURO NACIONAL,MUNICIPIO DE SAO FRANCISCO DE ITABAPOANA,OF ESPORTE 641 DE 28/12/2018 CONTRATO: CR.NR...,Sem processo,DESPESAS DE CAPITAL,INVESTIMENTOS,TRANSFERENCIAS A MUNICIPIOS,AUXILIOS - FUNDO A FUNDO,204750.00,204750.00,1.0
4,02/01/2019,Ordem Bancária,OBS DE OUTRAS CONTAS PARA CONTA UNICA,NÃO,MINISTERIO DA EDUCACAO,"INST.FED.DE EDUC.,CIENC.E TEC.DA BAHIA",INST. FED. DA BAHIA/CAMPUS SEABRA,"INST.FED.DE EDUC.,CIENC.E TEC.DA BAHIA",ALICIA VICTORIA ANJOS LIMA,12 - CANCELAMENTO PARCIAL DA 2018OB800770 - 20...,-8,Sem informação,Sem informação,Sem informação,Sem informação,-100.00,-100.00,1.0


#### 1. Gastos

Vamos tentar tirar todas as príncipais características de um gastos do governo.

In [3]:
# Agrupando a soma dos gastos Extraorçamentario e Orçamentario.
dic = {}
tipo = df['Extraorcamentario'].unique()
for i in tipo:
    sep = df.query('Extraorcamentario == "{}"'.format(i))
    if i == 'SIM':
        dic['Extraorçamentaria'] = sep['Valor_original_do_pagamento'].sum()
    elif i == 'NÃO':
        dic['Orçamentaria'] = sep['Valor_original_do_pagamento'].sum()

despesa_extra = pd.DataFrame({'Tipo': list(dic.keys()),'Valor_pago': list(dic.values())})
despesa_extra

,Tipo,Valor_pago
0,Orçamentaria,3.088074e+11
1,Extraorçamentaria,3.693431e+11


In [4]:
# Para melhorar o entendimento dessas análises, vamos exibir essas comparações de forma gráfica.
bar = go.Bar(x = despesa_extra['Tipo'],
             y = despesa_extra['Valor_pago'],
             marker = {'color': '#30336b'},
             opacity=0.4)

layout = go.Layout(title = 'Gráfico de comparação das despesas públicas.',
                   xaxis = {'title': 'Tipo de despesa'},
                   yaxis = {'title': 'Valor pago em R$'},
                   barmode = 'stack')

data = [bar]
fig = go.Figure(data = data, layout=layout)
py.iplot(fig)

Uma breve explicação sobre esses tipos de despesa:
* **Orçamentária**: É aquela fixada no orçamento público, logo, para sua realização depende de autorização legislativa.
* **Extraorçamentária**: São aquelas não previstas no orçamento. Sua execução independe de autorização legislativa. Correspondem à restituição ou entrega de valores arrecadados sob o título de receita extra-orçamentária

#### 2. Orgão superior com maior número de despesas.

In [5]:
# Separando em grupos para facilitar a comparação de qual será o ministério que mais teve despesas.
despesa_ministerios = df.groupby(['Orgao_superior'])[['Valor_original_do_pagamento']].sum()

# Colocando em ordem decrescente os gastos dos ministérios.
despesa_ministerios = despesa_ministerios.sort_values('Valor_original_do_pagamento', ascending = False)
despesa_ministerios.head()

,Valor_original_do_pagamento
Orgao_superior,
MINISTERIO DA ECONOMIA,6.002459e+11
MINISTERIO DA PREVIDENCIA SOCIAL,4.202331e+10
MINISTERIO DA EDUCACAO,1.192133e+10
MINISTERIO DA DEFESA,7.190943e+09
MINISTERIO DA SAUDE,3.739030e+09


In [6]:
# Plotando o gráfico dos 5 ministerios com maior número de gastos
data2 = [go.Bar(x = despesa_ministerios.index[:5].str.replace('MINISTERIO','M.'),
                y = despesa_ministerios['Valor_original_do_pagamento'][:5],
                marker=dict(
                color='rgb(158,202,225)',
                line=dict(
                    color='rgb(8,48,107)',
                    width=1.5),
                ),
                opacity=0.6)]

layout2 = go.Layout(title = 'Gráfico dos 5 orgãos com maior despesa em janeiro/19.',
                   xaxis = {'title': 'Ministérios'},
                   yaxis = {'title': 'Valor pago em R$'},
                   barmode = 'stack')

fig2 = go.Figure(data = data2, layout=layout2)
py.iplot(fig2)

#### 2.1. Despesas diárias do minitério da economia.

In [7]:
# Selecionando o ministerio e pegando apenas seus gastos diarios.
dme = df.query('Orgao_superior == "MINISTERIO DA ECONOMIA"')
dme_valor = dme.groupby('Data_emissao')[['Valor_original_do_pagamento']].sum()
#mdd = df.query('Orgao_superior != "MINISTERIO DA ECONOMIA"').groupby('Data_emissao')[['Valor_original_do_pagamento']].sum()
#mdd
dme.shape

(47668, 18)

In [8]:
# Plotando a curva de gastos diárias do ministério.
trace1 = go.Scatter(x = dme_valor.index,
                   y = dme_valor['Valor_original_do_pagamento'],
                   )
#trace2 = go.Scatter(x = mdd.index,
#                   y = mdd['Valor_original_do_pagamento'],
#                   )

data3 = [trace1]
py.iplot(data3)

Para entender melhor esses gastos, vamos exibir os tipos de despesas e seus gastos envolvidos em uma tabela.

In [17]:
# Pegando apenas o ministério da economia e tentando entender seus gastos.
test = dme.groupby(['Favorecido'])[['Valor_original_do_pagamento']].sum()
favorecido = test.sort_values('Valor_original_do_pagamento',ascending = False)
favorecido.head()

,Valor_original_do_pagamento
Favorecido,
RESGATE DE APLIC. DISPONIB. TN - ADM. DIV. PUBL,2.146128e+11
PAGAMENTO DA DIVIDA INTERNA,1.993830e+11
RECEBIMENTO DE EMISSÃO DE TITULOS,6.648115e+10
PAGAMENTOS DO INSS AO TESOURO NACIONAL,3.850480e+10
FUNDOS DE PARTICIPACAO - FPM E FPE,1.917077e+10


In [32]:
# Retornando os tipos de despesas que ocorreram nesse mês.
#invest = df.query('Grupo_de_despesa == "INVESTIMENTOS"')
#invest.head()
investimento = df.groupby(['Grupo_de_despesa'])[['Valor_original_do_pagamento']].sum()
investimento = investimento.sort_values('Valor_original_do_pagamento',ascending = False)
investimento

,Valor_original_do_pagamento
Grupo_de_despesa,
Sem informação,3.688955e+11
Múltiplo,2.086571e+11
OUTRAS DESPESAS CORRENTES,8.230415e+10
PESSOAL E ENCARGOS SOCIAIS,1.307199e+10
JUROS E ENCARGOS DA DIVIDA,2.810715e+09
INVESTIMENTOS,1.115551e+09
INVERSOES FINANCEIRAS,9.448319e+08
AMORTIZACAO/REFINANCIAMENTO DA DIVIDA,3.506320e+08
